# PCAによる次元削減（1280 → 50）

## 1. ライブラリのインポート

In [2]:
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import IncrementalPCA
from tqdm import tqdm

## 2. データの読み込み

In [3]:
model_name = "esm1b"

data_dir = f"../data/embedding-vectors/{model_name}/base"  # 1280次元のデータの読み込み
pca_components = 50  # 50次元にまで落とす
batch_rows = 50000  # partial_fit に投入する 行数 の目安（調整可）

### 2.1 ファイル一覧の取得

In [4]:
files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".npy")]  # .npy ファイルを全て取得
files.sort()  # 一応ソート

## 3. StandardScaler を バッチで学習（float32）

In [6]:
scaler = StandardScaler()

for filename in tqdm(files):
    arr = np.load(filename, mmap_mode="r")  # mmap_mode により 一気に読み込むことを防止できる

    # arr shape: (L, 1280)
    n = arr.shape[0]  # 配列長を取得
    start = 0

    while start < n:
        end = min(n, start + batch_rows)
        chunk = arr[start:end].astype(np.float32)
        scaler.partial_fit(chunk)
        start = end

100%|██████████████████████████████████████████████████████████████████████████████| 7716/7716 [00:22<00:00, 347.48it/s]


## 4. IncrementalPCA をバッチで学習

In [7]:
ipca = IncrementalPCA(n_components=pca_components)

for filename in tqdm(files):
    arr = np.load(filename, mmap_mode="r")  # 上と同様に
    n = arr.shape[0]
    start = 0

    while start < n:
        end = min(n, start + batch_rows)
        chunk = arr[start:end].astype(np.float32)
        chunk = (chunk - scaler.mean_) / np.sqrt(scaler.var_ + 1e-8)
        ipca.partial_fit(chunk)
        start = end

100%|█████████████████████████████████████████████████████████████████████████████| 7716/7716 [2:44:26<00:00,  1.28s/it]


## 5. 各配列を変換して保存（PCA済みを保管）

In [8]:
output_dir = f"../data/embedding-vectors/{model_name}/pca50"
os.makedirs(output_dir, exist_ok=True)

for filename in tqdm(files):
    arr = np.load(filename, mmap_mode="r").astype(np.float32)

    # 逐次的に transform
    n = arr.shape[0]
    out = np.zeros((n, pca_components), dtype=np.float32)
    start = 0
    idx = 0

    while start < n:
        end = min(n, start + batch_rows)
        chunk = arr[start:end]
        chunk = (chunk - scaler.mean_) / np.sqrt(scaler.var_ + 1e-8)
        out[idx: idx + (end - start), :] = ipca.transform(chunk)
        start = end
        idx += (end - start)

    base = os.path.basename(filename).replace(".npy", "")
    np.save(os.path.join(output_dir, base + "_pca50.npy"), out)

100%|███████████████████████████████████████████████████████████████████████████████| 7716/7716 [01:29<00:00, 86.56it/s]
